# Load Data

In [2]:
## Importing libraries

import pandas as pd
import numpy as np
from tqdm import tqdm
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import pandas as pd
train=pd.read_csv('/content/drive/MyDrive/Data Science Project - Team D/data/raw-data/train.csv')
train.describe().apply(lambda s: s.apply('{0:.5f}'.format))

test=pd.read_csv('/content/drive/MyDrive/Data Science Project - Team D/data/raw-data/test.csv')
test.describe().apply(lambda s: s.apply('{0:.5f}'.format))

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,200000.00000,200000.00000,200000.00000,200000.00000,200000.00000,200000.00000,200000.00000,200000.00000,200000.00000,200000.00000,...,200000.00000,200000.00000,200000.00000,200000.00000,200000.00000,200000.00000,200000.00000,200000.00000,200000.00000,200000.00000
mean,10.65874,-1.62424,10.70745,6.78821,11.07640,-5.05056,5.41516,16.52914,0.27713,7.56941,...,3.18977,7.45827,1.92594,3.32202,17.99697,-0.13366,2.29090,8.91243,15.86918,-3.24634
std,3.03672,4.04051,2.63389,2.05272,1.61646,7.86929,0.86469,3.42448,3.33337,1.23187,...,4.55124,3.02519,1.47997,3.99560,3.14065,1.42968,5.44635,0.92090,3.00872,10.39859
min,0.18870,-15.04340,2.35520,-0.02240,5.48440,-27.76700,2.21640,5.71370,-9.95600,4.24330,...,-14.09330,-2.40700,-3.34090,-11.41310,9.38280,-4.91190,-13.94420,6.16960,6.58400,-39.45780
25%,8.44298,-4.70012,8.73560,5.23050,9.89108,-11.20140,4.77260,13.93390,-2.30390,6.62380,...,-0.09500,5.16650,0.88298,0.58760,15.63478,-1.16070,-1.94860,8.26008,13.84727,-11.12400
50%,10.51380,-1.59050,10.56070,6.82235,11.09975,-4.83410,5.39160,16.42270,0.37200,7.63200,...,3.16240,7.37900,1.89260,3.42850,17.97760,-0.16200,2.40360,8.89280,15.94340,-2.72595
75%,12.73960,1.34340,12.49503,8.32760,12.25340,0.94257,6.00580,19.09455,2.93003,8.58482,...,6.33648,9.53110,2.95600,6.17420,20.39173,0.83790,6.51980,9.59590,18.04520,4.93540
max,22.32340,9.38510,18.71410,13.14200,16.03710,17.25370,8.30250,28.29280,9.66550,11.00360,...,20.35900,16.71650,8.00500,17.63260,27.94780,4.54540,15.92070,12.27580,26.53840,27.90740


# Prepare Data

In [ ]:
from tqdm import tqdm

col_names = [f"var_{i}" for i in range(200)]
for col in tqdm(col_names):
    count = test[col].value_counts()
    uniques = count.index[count == 1]
    test[col + "_u"] = test[col].isin(uniques)

test["unique"] = test[[col + "_u" for col in col_names]].any(axis=1)
test["unique"].sum()


real_test = test.loc[test["unique"], ["ID_code"] + col_names]
fake_test = test.loc[~test["unique"], ["ID_code"] + col_names]
train_and_test = pd.concat([train, real_test], axis=0)

for col in tqdm(col_names):
    count = train_and_test[col].value_counts().to_dict()
    train_and_test[col+"_unique"] = train_and_test[col].apply(
        lambda x: 1 if count[x] == 1 else 0).values
    fake_test[col+"_unique"] = 0 


real_test = train_and_test[train_and_test["ID_code"].str.contains("test")].copy()
real_test.drop(["target"], axis=1, inplace=True)
train = train_and_test[train_and_test["ID_code"].str.contains("train")].copy()
test = pd.concat([real_test, fake_test], axis=0)




# Split and normalize data

In [ ]:
#shuffle data
#train = train.sample(frac=1)

X = train.drop(['target', 'ID_code'], axis=1)
y=train[['target']]

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 
X = scaler.fit_transform(X) 

from sklearn.model_selection import train_test_split
# split into 90% train, 10% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [7]:
#shuffle data
#train = train.sample(frac=1)

X = train.drop(['ID_code'], axis=1)
y=train[['target']]

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 
X = scaler.fit_transform(X) 

from sklearn.model_selection import train_test_split
# split into 90% train, 10% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

# Pycaret model exploration

In [14]:
from pycaret.classification import *
s = setup(data, target = 'Class variable')

ImportError: ignored

In [11]:
# load dataset
from pycaret.datasets import get_data
diabetes = get_data('diabetes')

# init setup
from pycaret.classification import *
clf1 = setup(data = diabetes, target = 'Class variable')

# compare models
best = compare_models()

,Number of times pregnant,Plasma glucose concentration a 2 hours in an oral glucose tolerance test,Diastolic blood pressure (mm Hg),Triceps skin fold thickness (mm),2-Hour serum insulin (mu U/ml),Body mass index (weight in kg/(height in m)^2),Diabetes pedigree function,Age (years),Class variable
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


ImportError: ignored

In [ ]:
# load dataset
from pycaret.datasets import get_data
diabetes = get_data('diabetes')